# Simulating a one-dimensional flow along a Length and in time

 - taking a certain velocity and a certain length.
 - defining an initial condition.
 - solving ode to get time and space responses.
 - velocity reconstruction.
 - phase-shift detection methods.
 - solve for changing velocity and length.

In [1]:
import vtk
import numpy as np
from scipy.integrate import odeint